# refget-py tutorial

In [1]:
import refget
from refget import trunc512_digest

Show some results for sequence digests:

In [2]:
trunc512_digest('ACGT')

'68a178f7c740c5c240aa67ba41843b119d3bf9f8b0f0ac36'

In [3]:
trunc512_digest('TCGA')

'3912dddce432f3085c6b4f72a644c4c4c73f07215a9679ce'

In [4]:
trunc512_digest('ACGT', 26)

'68a178f7c740c5c240aa67ba41843b119d3bf9f8b0f0ac36cf70'

## Use a database

Now, instantiate a RefDB object. You have to provide a database where you will store lookup values. For a demo, you can also use a basic dictionary as a lookup database, but this will obviously not persist. 

Seed our database with a few pre-existing entries:

In [5]:
local_lookup_dict = {
    trunc512_digest('ACGT'): "ACGT",
    trunc512_digest('TCGA'): "TCGA"
}

rgdb_local = refget.RefDB(local_lookup_dict)


Retrieve sequences using the checksum

In [6]:
rgdb_local.refget(trunc512_digest('TCGA'))

'TCGA'

We can also add new sequences into the database:

In [7]:
rgdb_local.refget(trunc512_digest('TCGATCGA'))  # This sequence is not found in our database yet

'Not found'

In [8]:
checksum = rgdb_local.load_seq("TCGATCGA")  # So, let's add it into database

In [9]:
rgdb_local.refget(checksum)  # This time it returns

'TCGATCGA'

## Switching to a Redis back-end
Using a dict as a database will not persist. Let's instead use a redis back-end. If you're running a local redis server, you can use that as a back-end. First, start up a server like this:

```
docker run --rm --network='host' --workdir="`pwd`" redis:5.0.5 redis-server
```

 Then you can instantiate a new RefDB object that uses it like this:

In [10]:
rgdb = refget.RefDB(refget.RedisDict())

## Database insertion

Insert a sequence into the database, then retrieve it via checksum

In [11]:
checksum = rgdb.load_seq("GGAA")
rgdb.refget(checksum)

'GGAA'

## Insert and retrieve a sequence collection (fasta file)

In [12]:
fa_file = "demo_fasta/demo.fa"
checksum, content = rgdb.load_fasta(fa_file)

Here we retrieve all the sequences in the fasta file:

In [13]:
rgdb.refget(checksum)

OrderedDict([('chr1', 'ACGT'), ('chr2', 'TCGA')])

If you want it in fasta format there's a helper function for that:

In [14]:
print(rgdb.fasta_fmt(rgdb.refget(checksum)))

>chr1
ACGT
>chr2
TCGA


You can limit recursion to get just the checksums for individual sequences, rather than the sequences themselves:

In [15]:
rgdb.refget(checksum, reclimit=1)

OrderedDict([('chr1', '68a178f7c740c5c240aa67ba41843b119d3bf9f8b0f0ac36'),
             ('chr2', '3912dddce432f3085c6b4f72a644c4c4c73f07215a9679ce')])

The individual sequences are also retrievable independently because each sequence from the fasta file is stored as a primary unit. Test some single-sequence lookups from the database:

In [16]:
rgdb.refget(content["chr1"])

'ACGT'

In [17]:
rgdb.refget(trunc512_digest('ACGT'))

'ACGT'

Now if we kill that object and create a new object using the same redis back-end, the data persists because it's stored in the redis back-end:

In [18]:
rgdb = None
rgdb = refget.RefDB(refget.RedisDict())
rgdb.refget(checksum)

OrderedDict([('chr1', 'ACGT'), ('chr2', 'TCGA')])

## Comparing sequence collections

We may be interested in if collections have the same sequences in different order, or with different names. The `compare` function can provide this information. Let's load an example that adds another sequence:

In [19]:
fa_file = "demo_fasta/demo2.fa"
checksum2, content2 = rgdb.load_fasta(fa_file)

In [20]:
rgdb.refget(checksum2)

OrderedDict([('chr1', 'ACGT'), ('chr2', 'TCGA'), ('chrX', 'TTCCGGAA')])

In [21]:
rgdb.refget(checksum)

OrderedDict([('chr1', 'ACGT'), ('chr2', 'TCGA')])

In [23]:
rgdb.compare(checksum, checksum2)

'A is a sequence-level subset of B'

In [45]:
fa_file = "demo_fasta/demo3.fa"
checksum3, content3 = rgdb.load_fasta(fa_file)
rgdb.refget(checksum3)

OrderedDict([('chr1', 'ACGT'), ('chrX', 'TTCCGGAA')])

In [46]:
rgdb.compare(checksum, checksum3)

'A and B share some sequences'

In [47]:
fa_file = "demo_fasta/demo4.fa"
checksum4, content4 = rgdb.load_fasta(fa_file)
rgdb.refget(checksum4)

OrderedDict([('chrX', 'TTCCGGAA')])

In [48]:
rgdb.compare(checksum, checksum4)

'No sequences shared'

In [34]:
rgdb.compare(checksum2, checksum4)

'B is a sequence-level subset of A'

In [49]:
fa_file = "demo_fasta/demo5.fa"
checksum5, content5 = rgdb.load_fasta(fa_file)
rgdb.refget(checksum5)

OrderedDict([('chrX', 'TTCCGGAA'), ('chr1', 'ACGT')])

In [50]:
rgdb.compare(checksum, checksum5)

'A and B share some sequences'

In [51]:
rgdb.compare(checksum3, checksum5)

'Sequence-level identical, order mismatch'

In [52]:
fa_file = "demo_fasta/demo6.fa"
checksum6, content6 = rgdb.load_fasta(fa_file)
rgdb.refget(checksum6)

OrderedDict([('1', 'ACGT'), ('2', 'TCGA'), ('X', 'TTCCGGAA')])

In [53]:
rgdb.compare(checksum2, checksum6)

'Sequence-level identical, names mismatch'

# 3-layer refget

Yes, you can also continue this recursion indefinitely. To demonstrate, let's make a sequence that consists of these 2 fasta files checksums, and load that into the database.

In [24]:
layer3seq = "demo1:{};demo2:{}".format(checksum, checksum2)
layer3seq

'demo1:fa7f432a0ca2a8bb224562f1c2f091e37937c4b0a5f5acf7;demo2:6f2e5266e233ef82ae4fa15fb621ed2843b19cbb5f099391'

In [25]:
rgdb.load_seq(layer3seq)

'96b06cb2ef1fd4d8efb6c1d3a2be2415330c8fa77ebafc9d'

Can we retreieve it? You bet! At any recursion level:

In [26]:
rgdb.refget("96b06cb2ef1fd4d8efb6c1d3a2be2415330c8fa77ebafc9d")

OrderedDict([('demo1', OrderedDict([('chr1', 'ACGT'), ('chr2', 'TCGA')])),
             ('demo2',
              OrderedDict([('chr1', 'ACGT'),
                           ('chr2', 'TCGA'),
                           ('chrX', 'TTCCGGAA')]))])

In [27]:
rgdb.refget("96b06cb2ef1fd4d8efb6c1d3a2be2415330c8fa77ebafc9d", reclimit=1)

OrderedDict([('demo1', 'fa7f432a0ca2a8bb224562f1c2f091e37937c4b0a5f5acf7'),
             ('demo2', '6f2e5266e233ef82ae4fa15fb621ed2843b19cbb5f099391')])

In [28]:
rgdb.refget("96b06cb2ef1fd4d8efb6c1d3a2be2415330c8fa77ebafc9d", reclimit=2)

OrderedDict([('demo1',
              OrderedDict([('chr1',
                            '68a178f7c740c5c240aa67ba41843b119d3bf9f8b0f0ac36'),
                           ('chr2',
                            '3912dddce432f3085c6b4f72a644c4c4c73f07215a9679ce')])),
             ('demo2',
              OrderedDict([('chr1',
                            '68a178f7c740c5c240aa67ba41843b119d3bf9f8b0f0ac36'),
                           ('chr2',
                            '3912dddce432f3085c6b4f72a644c4c4c73f07215a9679ce'),
                           ('chrX',
                            'ef7dd6215b2dabc81d0a5eb0f0d84a739b0674de92679f88')]))])